In [1]:
import pandas as pd
from sklearn.datasets import load_files
import numpy as np
import math

In [2]:
DEBUG = False
# Switch this variable to debug the code

In [3]:
#Loading data from the folder
files = load_files('D:/20_newsgroups/')

In [4]:
len(files.data)

19997

In [5]:
if DEBUG:
    print(files)

In [6]:
if DEBUG:
    print(files.data[0])

In [7]:
import string
from nltk.corpus import stopwords
#For Punctuations and Stopwords

import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#For Lemmatizing

In [8]:
n = len(files.data)
# No. of Rows 

In [9]:
data = []
for i in range(n):
    data.append(files.data[i].decode("ISO-8859-1"))
# convert all byte type data

In [10]:
if DEBUG:
    print(data[0])

In [11]:
stop = stopwords.words('english')
# List of some english stopwords

punct = list(string.punctuation)
# List of punctuations

In [12]:
split = punct
split += " "
split += "\n"
num = ["1","2","3","4","5","6","7","8","9","0"]
split += num

# split contains all possible breaking point for tokenizing the sentence

In [13]:
def self_tokenize(str):
    # str is a paragraph of string type
    
    res=[]
    m = len(str)
    i=0
    temp = ""
    while i<m :
        if ((str[i] in split) or (i==m-1)):
            # if we reach the end of paragraph or we find a breaking point the stop here and add that word(if its length is greater than 1) in our res
            if len(temp)>1:
                res.append(temp)
            temp=""
            # Make temp empty for next word
        else:
            temp += str[i]
            #else add the current character in temp
            
        i+=1
    return res
# returning a list of tokenized words

In [14]:
if DEBUG:
    self_tokenize(data[0])

In [15]:
documents = []
for i in range(n):
    documents.append((self_tokenize(data[i]),files.target[i]))
    # append the tokenized list along with its corresponding target value

In [16]:
# function to get pos tag in required format for lemmatization
def get_simple_pos(w):
    if w.startswith("J"):
        return wordnet.ADJ
    elif w.startswith("N"):
        return wordnet.NOUN
    elif w.startswith("V"):
        return wordnet.VERB
    elif w.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
lemma = WordNetLemmatizer()

In [18]:
def clean_review(word):
    out = []
    # out will contain lemmatized words which are not present in stopwords
    for w in word:
        #if that word in not in stopword list
        if w.lower() not in stop:
            #get current pos tag
            cur_pos = pos_tag([w])[0][1]
            #lemmatize it
            w = lemma.lemmatize(w,pos=get_simple_pos(cur_pos))
            #add it to the output
            out.append(w.lower())
    return out

In [19]:
if DEBUG:
    clean_review(documents[0][0])

In [20]:
documents = [(clean_review(x),y) for x,y in documents]
# clean all the data that you are having

In [21]:
all_words = []
for i in range(n):
    all_words += documents[i][0]
# all_word contain all the words present in the files

In [22]:
if DEBUG:
    len(all_words)

In [23]:
freq = nltk.FreqDist(all_words)
#This will create a dictionary of words along with its freaquency in files

In [24]:
common = freq.most_common(3000)
#selecting the most common 3000 words

In [25]:
if DEBUG:
    print(common)

In [26]:
feature = [x for x,y in common]
#get all the features from the pairs
if DEBUG:
    print(feature)

In [27]:
lookup ={}
i=0
for x in feature:
    lookup[x]=i
    i=i+1
# maintain a dictionary of features along with its column number
#this will be useful when we will fill the array

In [28]:
mat = np.zeros((n,3000), dtype=int)
# create an array of size n X 3000
#where n ni no of rows

In [29]:
for cur in range(n):
    for word in documents[cur][0]:
        if word in lookup:
            mat[cur][lookup[word]]+=1
#filing the matrix 

In [30]:
X = mat
Y = files.target
# Assigning X and Y values

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=1)

# CLASSIFICATION USING INBUILT MULTINOMIAL NAIVE BAYES

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
# Using grid search to find optimal value of alpha

In [33]:
clf = MultinomialNB()


In [34]:
grid = {"alpha":[0.01,0.05,0.1,0.3,0.5,0.7,1,5,10]}
alg = GridSearchCV(clf,grid)
alg.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [35]:
alg.score(x_test,y_test)

0.8414

In [36]:
if DEBUG:
    print(alg.best_estimator_)

# IMPLEMENTING NAIVE BAYES FROM SCRATCH

In [37]:
def fit(x_train,y_train):
    #make a dictionary
    result = {}
    
    #store all possible class(output)
    all_class = set(y_train)
    
    for cur_class in all_class:
        # for each class create another dictionary with the riginal dictionary
        result[cur_class] = {}
        
        #Select all rows whose class is equal to current class
        find=(y_train==cur_class)
        
        #Hence, find the corresponding rows 
        cur_x = x_train[find]
        cur_y = y_train[find]
        
        # Now for each column store its count
        for cur_col in range(3000):
            result[cur_class][cur_col] = cur_x[:,cur_col].sum()
        
        #store the total count of the cur_class
        result[cur_class]["total_count"] = cur_x[:,:].sum()
        
    #find the total count of words of whole data
    result["total_data"]=x_train[:,:].sum()
    return result

In [54]:
def predict_single(X,dictionary):
    #it will predict the output of single row
    
    #all possible classes
    classes = set(dictionary.keys())
    
    #best_p => best probability till now
    #res => output corresponding to the best probability till now
    #first to check that in case it is the first iteration , assign it as the best_p 
    best_p=-1
    res=-1
    first=True
    
    for cur_class in classes:
        
        #Continue if cur_class is "total_data"
        if(cur_class=="total_data"):
            continue;
            
        #Initialise prob with the probability of cur_class i.e P(y==cur_class)
        prob = math.log(dictionary[cur_class]["total_count"]/dictionary["total_data"])
        
        for i in range(3000):
            
            #if the current word in feature is present in the testing data
            if(X[i]!=0):
                prob+=math.log(X[i]*(1+dictionary[cur_class][i])/(3000+dictionary[cur_class]["total_count"]))
                #we also give it a weightage according to the no. of time it is present in testing data
                #so multiply wuth X[i]
                
        if(first==True or best_p<prob):
            best_p=prob
            res=cur_class
            first=False
    return res;

In [47]:
def predict(x_test,dictionary):
    # it contains the output
    y_pred=[]
    
    for i in range(len(x_test)):
        #for all rows predict its output
        res=predict_single(x_test[i],dictionary)
        
        #append output to y_pred
        y_pred.append(res)
    return y_pred

In [40]:
#First generate the dictionary for quick prediction
dictionary=fit(x_train,y_train)

In [55]:
#predict the testing data
res=predict(x_test,dictionary)

In [56]:
#print the accuracy
print((res==y_test).sum()/len(y_test))

0.8508


### So we are getting almost same accuracy in both the case which is nearly equal to 0.85